In [1]:
%run Requirements.ipynb

24/06/05 11:46:05 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
24/06/05 11:46:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/05 11:46:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='MusicFusion'
)
cursor = conn.cursor()

Combined Music Artist Details

In [2]:
df = spark.sql(
    """
    SELECT DISTINCT SM.Artists
    FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic` AS AM
    INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/SpotifyMusic` AS SM 
    ON LTRIM(RTRIM(LOWER(AM.`Artist Name`))) = LTRIM(RTRIM(LOWER(SM.Artists)))
    LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/YoutubeMusic` AS YM
    ON LTRIM(RTRIM(LOWER(AM.`Artist Name`))) = LTRIM(RTRIM(LOWER(YM.`Artist Name`)))
    """
)

In [4]:
artists_df = df.select(F.col('Artists')).distinct()
artists_list = [row['Artists'] for row in artists_df.collect()]

news_api_key = News_API.get('API_KEY')

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('Mart/MusicFusion-Details')

Failed to fetch news for artist: Teddy Swims, Status Code: 429
Failed to fetch news for artist: Zach Bryan, Status Code: 429
Failed to fetch news for artist: Benson Boone, Status Code: 429
Failed to fetch news for artist: Noah Kahan, Status Code: 429
Failed to fetch news for artist: Sabrina Carpenter, Status Code: 429
Failed to fetch news for artist: Tommy Richman, Status Code: 429
Failed to fetch news for artist: Hozier, Status Code: 429
Failed to fetch news for artist: Chappell Roan, Status Code: 429
Failed to fetch news for artist: Eminem, Status Code: 429


24/06/05 11:30:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [4]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicFusion-Details')

columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS MusicFusionDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS MusicFusionDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO MusicFusionDetails VALUES {values};"
    cursor.execute(insert_query)

conn.commit()


Apple Music Artist Details

In [ ]:
df = spark.sql("SELECT DISTINCT * FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic`")

24/06/05 10:34:11 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [ ]:
artists_df = df.select(F.col('Artist Name')).distinct()
artists_list = [row['Artist Name'] for row in artists_df.collect()]

news_api_key = News_API.get('API_KEY')

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('Mart/AppleMusic-Details')
news_spark_df.write.mode('overwrite').format('parquet').saveAsTable('AppleMusicDetails')

24/06/05 10:35:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/06/05 10:35:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [5]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/AppleMusic-Details')

columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS AppleMusicDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS AppleMusicDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO AppleMusicDetails VALUES {values};"
    cursor.execute(insert_query)

conn.commit()


Spotify Artist Details

In [ ]:
df = spark.sql("SELECT DISTINCT * FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/SpotifyMusic`")

24/06/05 10:38:04 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [ ]:
artists_df = df.select(F.col('Artists')).distinct()
artists_list = [row['Artists'] for row in artists_df.collect()]

news_api_key = News_API.get('API_KEY')

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('Mart/Spotify-Details')
news_spark_df.write.mode('overwrite').format('parquet').saveAsTable('SpotifyDetails')

24/06/05 10:40:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/06/05 10:40:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [6]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/Spotify-Details')

columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS SpotifyDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS SpotifyDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO SpotifyDetails VALUES {values};"
    cursor.execute(insert_query)

conn.commit()
